<a href="https://colab.research.google.com/github/Jio-Kim/ML-DL/blob/main/Naive_Bayes(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 베르누이 나이브 베이즈를 활용한 스팸 분류

### 문제 정의
- 이메일 제목과 레이블 데이터를 활용해 베르누이 나이브 베이즈 분류로 스팸 메일 분류

In [2]:
# 필요한 파이썬 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

### 데이터 획득

In [3]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]
df = pd.DataFrame(email_list)

In [5]:
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


- email title : 이메일 제목
- spam : 스팸 메일 여부(True : 스팸 메일, False : 스팸 메일 아님)

### 데이터 다듬기
- 사이킷런의 베르누이 나이브 베이즈 분류기는 숫자만 다루므로 0과 1로 치환

In [7]:
df['label'] = df['spam'].map({True : 1, False : 0})

In [8]:
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [10]:
df_x = df['email title']
df_y = df['label']

- 베르누이 나이브 베이즈 입력 데이터는 고정된 크기의 벡터여야 함
  - 사이킷런의 CountVectorizer 이용하면 특정 데이터 안의 모든 단어를 포함한 고정 길이 벡터 생성 가능
  - binary = True 설정하면 이메일 제목에 특정 단어가 출현할 경우 무조건 1, 출현 안할경우 0 갖도록 설정

In [11]:
cv = CountVectorizer(binary = True)
x_traincv = cv.fit_transform(df_x)

In [12]:
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

중복된 단어가 있으면 1, 없으면 0

In [17]:
# 고정된 크기의 벡터에 포함된 단어 확인
cv.inverse_transform(encoded_input)

[array(['free', 'game', 'only', 'today'], dtype='<U9'),
 array(['cheapest', 'deal', 'flight'], dtype='<U9'),
 array(['limited', 'offer', 'only', 'time', 'today'], dtype='<U9'),
 array(['meeting', 'schedule', 'today'], dtype='<U9'),
 array(['attached', 'flight', 'schedule', 'your'], dtype='<U9'),
 array(['card', 'credit', 'statement', 'your'], dtype='<U9')]

In [19]:
# 고정된 벡터의 각 인덱스가 의미하는 단어 확인
list(cv.get_feature_names())

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

### 베르누이 나이브 베이즈 모델 학습하기

In [20]:
bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB()

### 테스트 데이터 다듬기

In [21]:
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1,False:0})
test_x=test_df["email title"]
test_y=test_df["label"]
x_testcv=cv.transform(test_x)

### 테스트

In [23]:
predictions = bnb.predict(x_testcv)
accuracy_score(test_y, predictions)

0.8333333333333334